In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [2]:
train_data = pd.read_csv("../data_prep/train_data.csv")
test_data = pd.read_csv("../data_prep/test_data.csv")

In [3]:
X_train = train_data.drop(['Activity','subject'],axis=1)
y_train = train_data['Activity']
X_test = test_data.drop(['Activity','subject'],axis=1)
y_test = test_data['Activity']
X_train.shape,y_train.shape,X_test.shape,y_test.shape

((240182, 12), (240182,), (103013, 12), (103013,))

In [4]:
# train_data = train_data.drop(columns=['subject'])
# test_data = test_data.drop(columns=['subject'])

In [5]:
# X_train = train_data.drop(columns=['Activity'])
# X_test = test_data.drop(columns=['Activity'])
# y_train = train_data['Activity']
# y_test = test_data['Activity']

In [6]:
from scipy import stats

#function to create time series datset for seuence modeling
def create_dataset(X, y, time_steps, step=1):
    Xs, ys = [], []
    for i in range(0, len(X) - time_steps, step):
        x = X.iloc[i:(i + time_steps)].values
        labels = y.iloc[i: i + time_steps]
        Xs.append(x)
        ys.append(stats.mode(labels)[0][0])
    return np.array(Xs), np.array(ys).reshape(-1, 1)

In [7]:
X_train,y_train = create_dataset(X_train, y_train, 100, step=50)
X_train.shape, y_train.shape

C:\Users\User\AppData\Local\Temp\ipykernel_35852\1935378417.py:10: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  ys.append(stats.mode(labels)[0][0])


((4802, 100, 12), (4802, 1))

In [8]:
X_test,y_test = create_dataset(X_test, y_test, 100, step=50)
X_test.shape, y_test.shape

C:\Users\User\AppData\Local\Temp\ipykernel_35852\1935378417.py:10: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  ys.append(stats.mode(labels)[0][0])


((2059, 100, 12), (2059, 1))

In [12]:
type(X_test)

numpy.ndarray

In [ ]:
# save train and test data
np.save('../train_test_data/X_train.npy', X_train)
np.save('../train_test_data/y_train.npy', y_train)
np.save('../train_test_data/X_test.npy', X_test)
np.save('../train_test_data/y_test.npy', y_test)

In [9]:
model = keras.Sequential()
model.add(layers.Input(shape=[100,12]))
model.add(layers.Conv1D(filters=32, kernel_size=3, padding="same"))
model.add(layers.BatchNormalization())
model.add(layers.ReLU())
model.add(layers.Conv1D(filters=64, kernel_size=3, padding="same"))
model.add(layers.BatchNormalization())
model.add(layers.ReLU())
model.add(layers.MaxPool1D(2))
model.add(layers.LSTM(64))
model.add(layers.Dense(units=128, activation='relu'))
model.add(layers.Dense(13, activation='softmax'))
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 100, 32)           1184      
                                                                 
 batch_normalization (Batch  (None, 100, 32)           128       
 Normalization)                                                  
                                                                 
 re_lu (ReLU)                (None, 100, 32)           0         
                                                                 
 conv1d_1 (Conv1D)           (None, 100, 64)           6208      
                                                                 
 batch_normalization_1 (Bat  (None, 100, 64)           256       
 chNormalization)                                                
                                                                 
 re_lu_1 (ReLU)              (None, 100, 64)           0

In [10]:
callbacks = [keras.callbacks.ModelCheckpoint("../models/best_model_lstm.h5", save_best_only=True, monitor="val_loss"),
             keras.callbacks.EarlyStopping(monitor="val_loss", patience=50, verbose=1)]
 
model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["sparse_categorical_accuracy"],)
 
model_history = model.fit(X_train,y_train, epochs= 10, validation_data=(X_test,y_test), callbacks=callbacks)

Epoch 1/10
151/151 [==============================] - 8s 31ms/step - loss: 0.7615 - sparse_categorical_accuracy: 0.7720 - val_loss: 0.7277 - val_sparse_categorical_accuracy: 0.7203
Epoch 2/10
  1/151 [..............................] - ETA: 5s - loss: 0.2614 - sparse_categorical_accuracy: 0.9375

C:\Users\User\AppData\Roaming\Python\Python38\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


151/151 [==============================] - 4s 26ms/step - loss: 0.2622 - sparse_categorical_accuracy: 0.9146 - val_loss: 0.2910 - val_sparse_categorical_accuracy: 0.8985
Epoch 3/10
151/151 [==============================] - 4s 24ms/step - loss: 0.1409 - sparse_categorical_accuracy: 0.9511 - val_loss: 0.1350 - val_sparse_categorical_accuracy: 0.9587
Epoch 4/10
151/151 [==============================] - 4s 24ms/step - loss: 0.1137 - sparse_categorical_accuracy: 0.9623 - val_loss: 0.4205 - val_sparse_categorical_accuracy: 0.8776
Epoch 5/10
151/151 [==============================] - 4s 24ms/step - loss: 0.1399 - sparse_categorical_accuracy: 0.9536 - val_loss: 0.0749 - val_sparse_categorical_accuracy: 0.9801
Epoch 6/10
151/151 [==============================] - 3s 23ms/step - loss: 0.1119 - sparse_categorical_accuracy: 0.9619 - val_loss: 0.0728 - val_sparse_categorical_accuracy: 0.9733
Epoch 7/10
151/151 [==============================] - 4s 25ms/step - loss: 0.0564 - sparse_categorical_acc

In [11]:
model = keras.models.load_model('../models/mhealth_best.h5')

train_loss, train_acc = model.evaluate(X_train,y_train)
test_loss, test_acc = model.evaluate(X_test,y_test)

print("Train accuracy", round(train_acc*100, 2),'%')
print("Train loss", train_loss)
print("Test accuracy", round(test_acc*100, 2),'%')
print("Test loss", test_loss)

65/65 [==============================] - 1s 8ms/step - loss: 0.0588 - sparse_categorical_accuracy: 0.9738
Train accuracy 99.04 %
Train loss 0.029633095487952232
Test accuracy 97.38 %
Test loss 0.05875091254711151
